In [13]:
import os

from dotenv import load_dotenv

load_dotenv()

True

In [16]:
# from langchain_community.document_loaders import GitLoader
from langchain_community.document_loaders import DirectoryLoader, TextLoader

def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")

# loader = GitLoader(
#     clone_url="https://github.com/langchain-ai/langchain",
#     repo_path="./langchain",
#     branch="master",
#     file_filter=file_filter,
# )
loader = DirectoryLoader(
    "./langchain",
    glob="**/*.mdx",
    loader_cls=TextLoader,
)

documents = loader.load()
print(len(documents))

409


In [17]:
for document in documents:
    document.metadata["filename"] = document.metadata["source"]

In [ ]:
import nest_asyncio
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset import TestsetGenerator
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from typing import List, Optional
from langchain_core.documents import Document
import logging

# 非同期処理用のネストを有効化
nest_asyncio.apply()

# エラーを記録するためのロガーを設定
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# HeadlineSplitterのエラーを処理するためのカスタムTestsetGenerator
class CustomTestsetGenerator(TestsetGenerator):
    def __init__(self, llm, embedding_model, **kwargs):
        super().__init__(llm=llm, embedding_model=embedding_model, **kwargs)

    def geneate_with_langchain_docs(self, documents: List[Document], testset_size: int = 4, context_size: Optional[int] = None):
        """
        LangChainドキュメントからテストセットを生成し、HeadlineSplitterエラーをスキップします

        Args:
            documents: 入力ドキュメントのリスト
            testset_size: 生成するテストケースの数
            context_size: コンテキストサイズ（省略可能）

        Returns:
            生成されたテストセット
        """
        # フィルター済みのドキュメントを格納するリスト
        filtered_documents = []

        # ドキュメントを処理し、エラーが発生したものをスキップ
        for doc in documents:
            try:
                # ドキュメントに最小限の検証を行う
                if hasattr(doc, 'page_content') and doc.page_content.strip():
                    filtered_documents.append(doc)
                else:
                    logger.warning(f"スキップしました: 空のコンテンツまたは無効なドキュメント形式")
            except Exception as e:
                logger.warning(f"ドキュメント処理中にエラーが発生したためスキップします: {str(e)}")

        logger.info(f"処理前のドキュメント数: {len(documents)}, 処理後: {len(filtered_documents)}")

        # 親クラスのメソッドを呼び出す前に名前を正しく修正
        # (タイプミスを修正: geneate_with_langchain_docs -> generate_with_langchain_docs)
        return super().generate_with_langchain_docs(
            filtered_documents,
            testset_size=testset_size,
            context_size=context_size
        )

# LLMおよび埋め込みモデルのインスタンス化
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

# カスタムTestsetGeneratorのインスタンス化
generator = CustomTestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)

# ドキュメントからテストセットを生成
# タイプミスを修正: geneate_with_langchain_docs -> generate_with_langchain_docs
try:
    testset = generator.generate_with_langchain_docs(
        documents,
        testset_size=4,
    )
    print(f"テストセット生成完了: {len(testset)} テストケースが生成されました")
except Exception as e:
    logging.error(f"テストセット生成中にエラーが発生しました: {str(e)}")
    # エラーの詳細情報を出力
    import traceback
    traceback.print_exc()

In [22]:
print(testset)

NameError: name 'testset' is not defined

In [ ]:
testset.to_pandas()

In [ ]:
from langsmith import Client

dataset_name = "agent-book"
client = Client()
if client.has_dataset(dataset_name=dataset_name):
    client.delete_dataset(dataset_name=dataset_name)

dataset = client.create_dataset(dataset_name=dataset_name)

In [ ]:
inputs = []
outputs = []
metadatas = []

for testset_record in testset.test_data:
   inputs.append(
       {
           "question": testset_record.question,
       }
   )
   outputs.append(
       {
           "contexts": testset_record.contexts,
           "ground_truth": testset_record.ground_truth,
       }
   )
   metadatas.append(
       {
           "source": testset_record.metadata[0]["source"],
           "evolution_type": testset_record.evolution_type,
       }
   )

In [ ]:
client.create_examples(
   inputs=inputs,
   outputs=outputs,
   metadata=metadatas,
   dataset_id=dataset.id,
)

In [ ]:
from typing import Any
from langchain_core.embeddings import Embeddings
from langchain_core.language_models import BaseChatModel
from langsmith.schemas import Example, Run
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper
from ragas.metrics.base import Metric, MetricWithEmbeddings, MetricWithLLM

class RagasMetricEvaluator:
    def __init__(self, metric: Metric, llm: BaseChatModel, embeddings: Embeddings):
        self.metric = metric
        # LLMとEmbeddingsをMetricに設定
        if isinstance(self.metric, MetricWithLLM):
            self.metric.llm = LangchainLLMWrapper(llm)
        if isinstance(self.metric, MetricWithEmbeddings):
            self.metric.embeddings = LangchainEmbeddingsWrapper(embeddings)

    def evaluate(self, run: Run, example: Example) -> dict[str, Any]:
        context_strs = [doc.page_content for doc in run.outputs["contexts"]]
        # Ragasの評価メトリクスのscoreメソッドでスコアを算出
        score = self.metric.score(
            {
                "question": example.inputs["question"],
                "answer": run.outputs["answer"],
                "contexts": context_strs,
                "ground_truth": example.outputs["ground_truth"],
            },
        )
        return {"key": self.metric.name, "score": score}

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.metrics import answer_relevancy, context_precision

metrics = [context_precision, answer_relevancy]
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
evaluators = [
   RagasMetricEvaluator(metric, llm, embeddings).evaluate
   for metric in metrics
]

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。
文脈: """
{context}
"""
質問: {question}
''')
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
retriever = db.as_retriever()
chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())

def predict(inputs: dict[str, Any]) -> dict[str, Any]:
    question = inputs["question"]
    output = chain.invoke(question)
    return {
        "contexts": output["context"],
        "answer": output["answer"],
    }

In [ ]:
from langsmith.evaluation import evaluate
evaluate(
    predict,
    data="agent-book",
    evaluators=evaluators,
)